In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from scipy.io import loadmat

%matplotlib inline
%load_ext autoreload
%autoreload 2

Data

In [2]:
data = torch.tensor(loadmat("./elevators.mat")["data"]).float()
X = data[:, :-1]
X = X - X.min(axis=0).values
X = 2 * (X / X.max(axis=0).values) - 1

y = data[:, -1]

In [3]:
n_train = int(0.8 * len(X))
X_train = X[:n_train].contiguous()
X_test = X[n_train:].contiguous()

y_train = y[:n_train].contiguous()
y_test = y[n_train:].contiguous()

In [4]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

In [5]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train, y_train, likelihood)

In [6]:
import os
training_iter = 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(X_train)
    # Calc loss and backprop gradients
    loss = -mll(output, y_train)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 0.785   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.742   lengthscale: 0.744   noise: 0.644
Iter 3/50 - Loss: 0.700   lengthscale: 0.797   noise: 0.598
Iter 4/50 - Loss: 0.660   lengthscale: 0.852   noise: 0.554
Iter 5/50 - Loss: 0.619   lengthscale: 0.908   noise: 0.513
Iter 6/50 - Loss: 0.578   lengthscale: 0.964   noise: 0.474
Iter 7/50 - Loss: 0.535   lengthscale: 1.021   noise: 0.437
Iter 8/50 - Loss: 0.494   lengthscale: 1.078   noise: 0.403
Iter 9/50 - Loss: 0.452   lengthscale: 1.134   noise: 0.370
Iter 10/50 - Loss: 0.409   lengthscale: 1.190   noise: 0.340
Iter 11/50 - Loss: 0.368   lengthscale: 1.245   noise: 0.312
Iter 12/50 - Loss: 0.323   lengthscale: 1.299   noise: 0.285
Iter 13/50 - Loss: 0.280   lengthscale: 1.351   noise: 0.261
Iter 14/50 - Loss: 0.236   lengthscale: 1.402   noise: 0.239
Iter 15/50 - Loss: 0.192   lengthscale: 1.451   noise: 0.218
Iter 16/50 - Loss: 0.148   lengthscale: 1.498   noise: 0.198
Iter 17/50 - Loss: 0.103   length

In [7]:
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(X_test))

In [8]:
rmse = torch.mean(torch.pow(observed_pred.mean - y_test, 2)).sqrt()
print(f"RMSE: {rmse.item()}")

RMSE: 0.09203086048364639
